In [1]:
import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.layers import BatchNormalization
from keras.applications import ResNet50
from keras.models import Model
from keras.layers import Input, Conv2DTranspose, Conv2D, UpSampling2D, BatchNormalization, Activation,MaxPooling2D, Dense,Add,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, Dense, Add
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size=3, strides=1):
    shortcut = x
    # First convolution layer
    x = Conv2D(filters, kernel_size, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # Second convolution layer
    x = Conv2D(filters, kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    
    # Shortcut connection
    if strides != 1 or shortcut.shape[-1] != filters:
        shortcut = Conv2D(filters, (1, 1), strides=strides, padding='valid')(shortcut)
        shortcut = BatchNormalization()(shortcut)
    
    x = Add()([x, shortcut])
    x = Activation('relu')(x)
    return x




def create_resnet(input_shape=(80, 160, 3), num_classes=1, num_filters=64, num_blocks=[2, 2, 2]):
    inputs = Input(shape=input_shape)
    
    # Initial convolution layer
    x = Conv2D(num_filters, (7, 7), strides=(2, 2), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # MaxPooling layer
    x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    # Residual blocks
    for block_num in range(len(num_blocks)):
        for layer_num in range(num_blocks[block_num]):
            strides = 1 if block_num == 0 and layer_num == 0 else 2
            x = residual_block(x, num_filters * (2 ** block_num), strides=strides)
    
    # Upsample the output to match the desired shape
    x = UpSampling2D(size=(5, 5))(x)  # Adjust the upsampling factor
    x = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)
    
    # Resize the output to the desired shape
    x = tf.image.resize(x, (80, 160), method='bilinear')
    
    model = Model(inputs, x)
    return model

In [5]:
 # Load training images
train_images = pickle.load(open("full_CNN_train.p", "rb" ))

    # Load image labels
labels = pickle.load(open("full_CNN_labels.p", "rb" ))

    # Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)

    # Normalize labels - training images get normalized to start in the network
labels = labels / 255

    # Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
    # Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.3)

    # Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 20  

    # Create the neural network
model = create_resnet(input_shape=(80, 160, 3), num_classes=1)

    # Using a generator to help the model use less data
    # Channel shifts help with shadows slightly
datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)

    # Compiling and training the model
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])
model.fit(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
epochs=epochs, verbose=1, validation_data=(X_val, y_val))

    # Freeze layers since training is done
model.trainable = False
model.compile(optimizer='adam', loss='mean_squared_error',metrics=['accuracy'])

    # Save model architecture and weights
model.save('full_Res_Net_new_model.keras')
    
    #save model architecture in json file
model_json = model.to_json()
with open("RES_NET_model.json", "w") as json_file:
    json_file.write(model_json)


Epoch 1/20
69/69 [==============================] - 200s 3s/step - loss: 0.1440 - accuracy: 0.8135 - val_loss: 0.1534 - val_accuracy: 0.8288
Epoch 2/20
69/69 [==============================] - 220s 3s/step - loss: 0.1282 - accuracy: 0.8291 - val_loss: 0.1423 - val_accuracy: 0.8288
Epoch 3/20
69/69 [==============================] - 208s 3s/step - loss: 0.1280 - accuracy: 0.8289 - val_loss: 0.1385 - val_accuracy: 0.8288
Epoch 4/20
69/69 [==============================] - 233s 3s/step - loss: 0.1280 - accuracy: 0.8291 - val_loss: 0.1286 - val_accuracy: 0.8288
Epoch 5/20
69/69 [==============================] - 223s 3s/step - loss: 0.1277 - accuracy: 0.8291 - val_loss: 0.1296 - val_accuracy: 0.8237
Epoch 6/20
69/69 [==============================] - 239s 3s/step - loss: 0.1279 - accuracy: 0.8291 - val_loss: 0.1306 - val_accuracy: 0.8288
Epoch 7/20
69/69 [==============================] - 237s 3s/step - loss: 0.1276 - accuracy: 0.8290 - val_loss: 0.1290 - val_accuracy: 0.8288
Epoch 8/20
69

In [6]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 80, 160, 3)]         0         []                            
                                                                                                  
 conv2d_38 (Conv2D)          (None, 40, 80, 64)           9472      ['input_3[0][0]']             
                                                                                                  
 batch_normalization_36 (Ba  (None, 40, 80, 64)           256       ['conv2d_38[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_26 (Activation)  (None, 40, 80, 64)           0         ['batch_normalization_36

 activation_32 (Activation)  (None, 5, 10, 128)           0         ['add_14[0][0]']              
                                                                                                  
 conv2d_47 (Conv2D)          (None, 3, 5, 128)            147584    ['activation_32[0][0]']       
                                                                                                  
 batch_normalization_45 (Ba  (None, 3, 5, 128)            512       ['conv2d_47[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_33 (Activation)  (None, 3, 5, 128)            0         ['batch_normalization_45[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_48

 a)                                                                                               
                                                                                                  
Total params: 2878017 (10.98 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 2878017 (10.98 MB)
__________________________________________________________________________________________________


In [12]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score

# Assuming you have already trained and compiled your model
# model = ...

# Assuming you have already loaded and preprocessed your validation data
import os
import pandas as pd
from PIL import Image

# Define the directory containing test images
test_images_dir = "E:\GitHub Desktop\Lane_Detection\Approach-2\test_images"

# List all image files in the directory
image_files = os.listdir(test_images_dir)

# Sort the image files if needed
image_files.sort()

# Load test labels from a CSV file (assuming it has a 'label' column)
test_labels_file = "E:\GitHub Desktop\Lane_Detection\Approach-2\test_predict"
test_labels_df = pd.read_csv(test_labels_file)

# Extract labels as a NumPy array
test_labels = test_labels_df['label'].values

# Initialize an empty list to store test images
test_images = []

# Load test images
for image_file in image_files:
    image_path = os.path.join(test_images_dir, image_file)
    image = Image.open(image_path)
    # You may need to preprocess the image (e.g., resizing, normalization) here
    test_images.append(np.array(image))

# Convert the list of test images to a NumPy array
test_images = np.array(test_images)

# Now you can use test_images and test_labels for evaluation


# Make predictions using your trained model
predictions = model.predict(X_val)

# Convert predictions to binary values based on a threshold (e.g., 0.5)
threshold = 0.5  # You can adjust this threshold as needed
binary_predictions = (predictions > threshold).astype(np.float32)

# Convert labels to binary values based on the same threshold
binary_labels = (y_val / 255 > threshold).astype(np.float32)

# Calculate F1 score, recall, and precision
f1 = f1_score(binary_labels, binary_predictions)
recall = recall_score(binary_labels, binary_predictions)
precision = precision_score(binary_labels, binary_predictions)

print("F1 Score:", f1)
print("Recall:", recall)
print("Precision:", precision)


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'E:\\GitHub Desktop\\Lane_Detection\\Approach-2\test_images'